In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, coalesce, avg, lit, desc
import os

In [50]:
spark = SparkSession.builder \
    .appName("COVID-19 Analysis") \
    .getOrCreate()

In [51]:
Month_Mapping = {
    'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04',
    'may': '05', 'jun': '06', 'jul': '07', 'aug': '08',
    'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'
}

In [52]:
Month_Abbrevation = input("Enter the Month Abbreviation (e.g., 'Jan' for January,): ").strip().lower()
year = input("Enter the year (YYYY): ").strip()

Enter the Month Abbreviation (e.g., 'Jan' for January,): jan
Enter the year (YYYY): 2021


In [53]:
Month_MM = Month_Mapping.get(Month_Abbrevation)
if not month_mm:
    print("Invalid month abbreviation.")
    exit()

In [54]:
folder_path = "csse_covid_19_daily_reports_us"

dataframes = []

for day in range(1, 32): 
    
    File_Name = f"{Month_MM}-{day:02d}-{year}.csv"
    File_Path = os.path.join(folder_path, File_Name)

In [55]:
if os.path.exists(File_Path):
        
        dataframe = spark.read.csv(File_Path, header=True, inferSchema=True)
        dataframe = dataframe.withColumn("IFR", (col("Deaths") / col("Confirmed")) * 100)
        dataframes.append(df)

In [56]:
if dataframes:
    
    january_df = dataframes[0]
    for df in dataframes[1:]:
        january_df = january_df.union(dataframe)

   
    Aggregated_dataframe = january_df.groupby("Province_State") \
                               .agg(sum("Deaths").alias("Total_Deaths"), 
                                    sum("Confirmed").alias("Total_Confirmed"))

   
    Aggregated_dataframe = Aggregated_dataframe.withColumn("IFR", (col("Total_Deaths") / col("Total_Confirmed")) * 100)

   
    top_10_states = Aggregated_dataframe.orderBy(desc("IFR")).limit(10)
    print(f"Top 10 States with Highest Infection Fatality Ratio (IFR) in {month_abbrev.capitalize()} {year}:")
    top_10_states.show()
else:
    print(f"No data available for {month_abbrev.capitalize()} {year}.")

Top 10 States with Highest Infection Fatality Ratio (IFR) in Jan 2021:
+--------------+------------+---------------+------------------+
|Province_State|Total_Deaths|Total_Confirmed|               IFR|
+--------------+------------+---------------+------------------+
|Grand Princess|           3|            103| 2.912621359223301|
|  Pennsylvania|       40581|        2661481|1.5247525719702677|
|   Mississippi|       10935|         727418|1.5032622233708817|
|    New Jersey|       31442|        2111856|1.4888325719177822|
|   Connecticut|       10010|         699400|1.4312267657992566|
|      Michigan|       32263|        2258465|1.4285366388232716|
|          Ohio|       36291|        2580405| 1.406407133763886|
|       Alabama|       17086|        1218257|1.4024955325518345|
|       Arizona|       26205|        1870644|1.4008544650933048|
|       Georgia|       32976|        2368052|1.3925369882080292|
+--------------+------------+---------------+------------------+



In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

In [58]:
states_to_color = [row['Province_State'] for row in top_10_states.collect()]

print("States to color:", states_to_color)
import plotly.graph_objects as go

States to color: ['Grand Princess', 'Pennsylvania', 'Mississippi', 'New Jersey', 'Connecticut', 'Michigan', 'Ohio', 'Alabama', 'Arizona', 'Georgia']


In [59]:
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

known_states_to_color = [state for state in states_to_color if state in state_abbr]

In [60]:
fig = go.Figure()


color_intensity = 1.2 
for state in known_states_to_color:
    fig.add_trace(go.Choropleth(
        locationmode='USA-states',
        locations=[state_abbr[state]],  
        z=[1],  
        #colorscale=[[0, f'rgba(0, 0, 255, {color_intensity})'], [1, f'rgba(0, 0, 255, {color_intensity})']],  
        colorscale = [[0, f'rgba(0, 255, 0, {color_intensity})'], [1, f'rgba(0, 255, 0, {color_intensity})']],
        showscale=False, 
    ))
    color_intensity -= 0.1

In [61]:
fig.update_layout(
    title_text='Map of US States',
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
    ),
)